In [151]:
#ne svidja mi se skup podataka
#mozda predjem na onaj sto ima na onom nekom stjtu sto je ona dala
#deluje mi jednostavnije
# a uspeo sam da skinem onaj

import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds #mozemo ukljuciti i nas svd

In [152]:
ratings = pd.read_csv('input/ratings_small.csv')

movies = pd.read_csv('input/movies_metadata.csv', usecols=['id', 'title', 'genres'])

In [153]:
ratings.dtypes
i = movies[movies['id'] == '1997-08-20'].index
ratings['movieId'] = ratings['movieId'].astype('str')

In [154]:
movies['id'] = movies['id'].astype('str')


In [155]:
#pravimo matricu koja ima jednu kolonu za userId i po kolonu za svaki film
Ratings = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
Ratings.head()


movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,...,99795,998,99811,99813,99839,99846,999,99912,99917,99992
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
#normalizovanje podataka
R = Ratings.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [157]:
#set upujemo svd
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [158]:
sigma = np.diag(sigma)

In [159]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [160]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,...,99795,998,99811,99813,99839,99846,999,99912,99917,99992
0,-0.054239,0.037670,-0.009920,-0.020544,-0.004722,-0.008621,-0.004933,-0.003897,0.019938,-0.012280,...,0.008019,-0.011968,-0.003747,-0.015609,-0.004994,-0.003580,0.028580,-0.004408,-0.005453,-0.003580
1,0.419835,2.220256,-0.097641,-0.002199,-0.000172,-0.001921,-0.006062,-0.057228,-0.005017,-0.017252,...,-0.035397,-0.036604,0.004896,0.035671,-0.006310,-0.000182,0.006596,0.003073,-0.009955,-0.000182
2,1.345619,0.141176,-0.005922,0.033891,-0.012645,0.014838,0.005164,-0.015414,0.012995,-0.045484,...,0.018898,0.044556,-0.026354,-0.023502,-0.004873,-0.001294,0.023425,0.023469,0.000569,-0.001294
3,1.133455,1.611664,-0.661506,0.026870,0.022393,-0.050393,-0.008704,0.146679,0.052567,0.057701,...,0.009655,0.439857,0.051705,-0.031724,0.037440,0.005385,-0.348629,-0.027301,-0.013167,0.005385
4,1.389578,0.307926,-0.135217,0.077073,-0.008298,-0.002522,-0.010923,0.001503,0.045325,-0.042554,...,-0.020408,0.045892,-0.003615,0.160520,-0.004087,0.010483,0.013108,0.013803,-0.052466,0.010483


In [161]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'id').
                     sort_values(['rating'], ascending=False)
                 )

#     print 'User {0} has already rated {1} movies.'.format(userID, user_full.shape[0])
#     print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['id'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'id',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [162]:
already_rated, predictions = recommend_movies(preds, 152, movies, ratings, 20)

In [163]:
# Top 20 movies that User 600 has rated 
already_rated.head(20)

,userId,movieId,rating,timestamp,genres,id,title
109,152,4027,5.0,1335949459,NaN,NaN,NaN
6,152,47,5.0,1335948042,NaN,NaN,NaN
159,152,31410,5.0,1335900450,"[{'id': 35, 'name': 'Comedy'}]",31410,My Sweet Little Village
76,152,1653,5.0,1335947506,"[{'id': 18, 'name': 'Drama'}]",1653,The Motorcycle Diaries
7,152,50,5.0,1335948008,NaN,NaN,NaN
54,152,1175,5.0,1335948656,NaN,NaN,NaN
5,152,32,5.0,1335947997,NaN,NaN,NaN
88,152,2571,5.0,1335948000,NaN,NaN,NaN
147,152,7502,5.0,1335900352,NaN,NaN,NaN
84,152,2028,5.0,1335948073,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",2028,Say Anything...


In [164]:
predictions

,genres,id,title,movieId
6102,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",6016,The Good Thief,6016
532,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",858,Sleepless in Seattle,858
6870,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",648,Beauty and the Beast,648
1320,"[{'id': 28, 'name': 'Action'}, {'id': 14, 'nam...",364,Batman Returns,364
23961,"[{'id': 99, 'name': 'Documentary'}]",2692,The Red Elvis,2692
3038,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",1213,The Talented Mr. Ripley,1213
1392,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",1968,Fools Rush In,1968
43280,"[{'id': 10769, 'name': 'Foreign'}]",48780,Boat,48780
1490,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",8874,My Best Friend's Wedding,8874
1889,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",165,Back to the Future Part II,165
